# IPL Match Prediction


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
match_df = pd.read_csv('matches.csv')
match_df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [3]:
deliveries_df = pd.read_csv('deliveries.csv')
deliveries_df.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [4]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               756 non-null    int64 
 1   season           756 non-null    int64 
 2   city             749 non-null    object
 3   date             756 non-null    object
 4   team1            756 non-null    object
 5   team2            756 non-null    object
 6   toss_winner      756 non-null    object
 7   toss_decision    756 non-null    object
 8   result           756 non-null    object
 9   dl_applied       756 non-null    int64 
 10  winner           752 non-null    object
 11  win_by_runs      756 non-null    int64 
 12  win_by_wickets   756 non-null    int64 
 13  player_of_match  752 non-null    object
 14  venue            756 non-null    object
 15  umpire1          754 non-null    object
 16  umpire2          754 non-null    object
 17  umpire3          119 non-null    ob

In [5]:
deliveries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179078 entries, 0 to 179077
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          179078 non-null  int64 
 1   inning            179078 non-null  int64 
 2   batting_team      179078 non-null  object
 3   bowling_team      179078 non-null  object
 4   over              179078 non-null  int64 
 5   ball              179078 non-null  int64 
 6   batsman           179078 non-null  object
 7   non_striker       179078 non-null  object
 8   bowler            179078 non-null  object
 9   is_super_over     179078 non-null  int64 
 10  wide_runs         179078 non-null  int64 
 11  bye_runs          179078 non-null  int64 
 12  legbye_runs       179078 non-null  int64 
 13  noball_runs       179078 non-null  int64 
 14  penalty_runs      179078 non-null  int64 
 15  batsman_runs      179078 non-null  int64 
 16  extra_runs        179078 non-null  int

## Exploratory Data Analysis

In [6]:
match_df['winner'].value_counts()

Mumbai Indians                 109
Chennai Super Kings            100
Kolkata Knight Riders           92
Royal Challengers Bangalore     84
Kings XI Punjab                 82
Rajasthan Royals                75
Delhi Daredevils                67
Sunrisers Hyderabad             58
Deccan Chargers                 29
Gujarat Lions                   13
Pune Warriors                   12
Delhi Capitals                  10
Rising Pune Supergiant          10
Kochi Tuskers Kerala             6
Rising Pune Supergiants          5
Name: winner, dtype: int64

In [7]:
match_df['team1'].value_counts()

Mumbai Indians                 101
Kings XI Punjab                 91
Chennai Super Kings             89
Royal Challengers Bangalore     85
Kolkata Knight Riders           83
Delhi Daredevils                72
Rajasthan Royals                67
Sunrisers Hyderabad             63
Deccan Chargers                 43
Pune Warriors                   20
Gujarat Lions                   14
Rising Pune Supergiant           8
Rising Pune Supergiants          7
Kochi Tuskers Kerala             7
Delhi Capitals                   6
Name: team1, dtype: int64

In [8]:
#check for missing values in winner column because we are going to predict winner here

match_df[match_df['winner'].isnull()==True]

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
300,301,2011,Delhi,2011-05-21,Delhi Daredevils,Pune Warriors,Delhi Daredevils,bat,no result,0,NaN,0,0,NaN,Feroz Shah Kotla,SS Hazare,RJ Tucker,NaN
545,546,2015,Bangalore,2015-04-29,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,no result,0,NaN,0,0,NaN,M Chinnaswamy Stadium,JD Cloete,PG Pathak,NaN
570,571,2015,Bangalore,2015-05-17,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore,field,no result,0,NaN,0,0,NaN,M Chinnaswamy Stadium,HDPK Dharmasena,K Srinivasan,NaN
744,11340,2019,Bengaluru,30/04/19,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,no result,0,NaN,0,0,NaN,M. Chinnaswamy Stadium,Nigel Llong,Ulhas Gandhe,Anil Chaudhary


In [9]:
match_df['winner'].fillna('Draw',inplace = True)

In [10]:
team_encodings = {
    
    'Mumbai Indians' : 1,
    'Kolkata Knight Riders': 2,
    'Royal Challengers Bangalore' : 3,
    'Deccan Chargers':4,
    'Chennai Super Kings': 5,
    'Rajasthan Royals' : 6,
    'Delhi Daredevils' : 7,
    'Gujarat Lions' : 8,
    'Kings XI Punjab':9,
    'Sunrisers Hyderabad': 10,
    'Rising Pune Supergiant': 11,
    'Rising Pune Supergiants' : 12,
    'Kochi Tuskers Kerala': 13,
    'Pune Warriors' : 14,
    'Delhi Capitals' : 15,
    'Draw' : 16
    
}

In [11]:
team_encode_dict = {
    
    'team1' : team_encodings,
    'team2' : team_encodings,
    'toss_winner': team_encodings,
    'winner' : team_encodings
}

In [12]:
team_encode_dict

{'team1': {'Mumbai Indians': 1,
  'Kolkata Knight Riders': 2,
  'Royal Challengers Bangalore': 3,
  'Deccan Chargers': 4,
  'Chennai Super Kings': 5,
  'Rajasthan Royals': 6,
  'Delhi Daredevils': 7,
  'Gujarat Lions': 8,
  'Kings XI Punjab': 9,
  'Sunrisers Hyderabad': 10,
  'Rising Pune Supergiant': 11,
  'Rising Pune Supergiants': 12,
  'Kochi Tuskers Kerala': 13,
  'Pune Warriors': 14,
  'Delhi Capitals': 15,
  'Draw': 16},
 'team2': {'Mumbai Indians': 1,
  'Kolkata Knight Riders': 2,
  'Royal Challengers Bangalore': 3,
  'Deccan Chargers': 4,
  'Chennai Super Kings': 5,
  'Rajasthan Royals': 6,
  'Delhi Daredevils': 7,
  'Gujarat Lions': 8,
  'Kings XI Punjab': 9,
  'Sunrisers Hyderabad': 10,
  'Rising Pune Supergiant': 11,
  'Rising Pune Supergiants': 12,
  'Kochi Tuskers Kerala': 13,
  'Pune Warriors': 14,
  'Delhi Capitals': 15,
  'Draw': 16},
 'toss_winner': {'Mumbai Indians': 1,
  'Kolkata Knight Riders': 2,
  'Royal Challengers Bangalore': 3,
  'Deccan Chargers': 4,
  'Chenn

In [13]:
match_df.replace(team_encode_dict,inplace = True)

In [14]:
#missing value in city column
match_df['city'].value_counts()

Mumbai            101
Kolkata            77
Delhi              74
Bangalore          66
Hyderabad          64
Chennai            57
Jaipur             47
Chandigarh         46
Pune               38
Durban             15
Bengaluru          14
Visakhapatnam      13
Ahmedabad          12
Centurion          12
Mohali             10
Rajkot             10
Dharamsala          9
Indore              9
Johannesburg        8
Cape Town           7
Ranchi              7
Cuttack             7
Port Elizabeth      7
Abu Dhabi           7
Sharjah             6
Raipur              6
Kochi               5
Kanpur              4
Nagpur              3
East London         3
Kimberley           3
Bloemfontein        2
Name: city, dtype: int64

In [15]:
match_df[match_df['city'].isnull()==True]

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
461,462,2014,NaN,2014-04-19,1,3,3,field,normal,0,3,0,7,PA Patel,Dubai International Cricket Stadium,Aleem Dar,AK Chaudhary,NaN
462,463,2014,NaN,2014-04-19,2,7,2,bat,normal,0,7,0,4,JP Duminy,Dubai International Cricket Stadium,Aleem Dar,VA Kulkarni,NaN
466,467,2014,NaN,2014-04-23,5,6,6,field,normal,0,5,7,0,RA Jadeja,Dubai International Cricket Stadium,HDPK Dharmasena,RK Illingworth,NaN
468,469,2014,NaN,2014-04-25,10,7,10,bat,normal,0,10,4,0,AJ Finch,Dubai International Cricket Stadium,M Erasmus,S Ravi,NaN
469,470,2014,NaN,2014-04-25,1,5,1,bat,normal,0,5,0,7,MM Sharma,Dubai International Cricket Stadium,BF Bowden,M Erasmus,NaN
474,475,2014,NaN,2014-04-28,3,9,9,field,normal,0,9,0,5,Sandeep Sharma,Dubai International Cricket Stadium,BF Bowden,S Ravi,NaN
476,477,2014,NaN,2014-04-30,10,1,1,field,normal,0,10,15,0,B Kumar,Dubai International Cricket Stadium,HDPK Dharmasena,M Erasmus,NaN


In [16]:
#only dubai stadiums doesnt have city name, so it is replaced with dubai

match_df['city'].fillna('Dubai',inplace = True)

In [17]:
match_df

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,10,3,3,field,normal,0,10,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,1,11,11,field,normal,0,11,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,8,2,2,field,normal,0,2,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,11,9,9,field,normal,0,9,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,3,7,3,bat,normal,0,3,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,11347,2019,Mumbai,05/05/19,2,1,1,field,normal,0,1,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi
752,11412,2019,Chennai,07/05/19,5,1,5,bat,normal,0,1,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould
753,11413,2019,Visakhapatnam,08/05/19,10,15,15,field,normal,0,15,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN
754,11414,2019,Visakhapatnam,10/05/19,15,5,5,field,normal,0,5,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin


In [18]:
match_df.describe()

,id,season,team1,team2,toss_winner,dl_applied,winner,win_by_runs,win_by_wickets
count,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000
mean,1792.178571,2013.444444,5.607143,5.670635,5.522487,0.025132,5.465608,13.283069,3.350529
std,3464.478148,3.366895,3.505591,3.557101,3.496189,0.156630,3.536022,23.471144,3.387963
min,1.000000,2008.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
25%,189.750000,2011.000000,3.000000,3.000000,2.000000,0.000000,2.000000,0.000000,0.000000
50%,378.500000,2013.000000,5.000000,5.000000,5.000000,0.000000,5.000000,0.000000,4.000000
75%,567.250000,2016.000000,9.000000,8.000000,8.000000,0.000000,8.000000,19.000000,6.000000
max,11415.000000,2019.000000,15.000000,15.000000,15.000000,1.000000,16.000000,146.000000,10.000000


###  Toss Wins vs Match Wins

In [19]:
toss_wins = match_df['toss_winner'].value_counts(sort=True)
toss_wins

1     98
2     92
5     89
9     81
3     81
7     80
6     80
10    46
4     43
14    20
8     15
15    10
13     8
12     7
11     6
Name: toss_winner, dtype: int64

In [20]:
match_wins = match_df['winner'].value_counts(sort = True)
match_wins

1     109
5     100
2      92
3      84
9      82
6      75
7      67
10     58
4      29
8      13
14     12
15     10
11     10
13      6
12      5
16      4
Name: winner, dtype: int64

In [21]:
for idx,val in toss_wins.iteritems():
    print(f"{list(team_encode_dict['team1'].keys())[idx-1]} -> {toss_wins[idx]}")

Mumbai Indians -> 98
Kolkata Knight Riders -> 92
Chennai Super Kings -> 89
Kings XI Punjab -> 81
Royal Challengers Bangalore -> 81
Delhi Daredevils -> 80
Rajasthan Royals -> 80
Sunrisers Hyderabad -> 46
Deccan Chargers -> 43
Pune Warriors -> 20
Gujarat Lions -> 15
Delhi Capitals -> 10
Kochi Tuskers Kerala -> 8
Rising Pune Supergiants -> 7
Rising Pune Supergiant -> 6


In [22]:
for idx,val in match_wins.iteritems():
    print(f"{list(team_encode_dict['winner'].keys())[idx-1]} -> {match_wins[idx]}")

Mumbai Indians -> 109
Chennai Super Kings -> 100
Kolkata Knight Riders -> 92
Royal Challengers Bangalore -> 84
Kings XI Punjab -> 82
Rajasthan Royals -> 75
Delhi Daredevils -> 67
Sunrisers Hyderabad -> 58
Deccan Chargers -> 29
Gujarat Lions -> 13
Pune Warriors -> 12
Delhi Capitals -> 10
Rising Pune Supergiant -> 10
Kochi Tuskers Kerala -> 6
Rising Pune Supergiants -> 5
Draw -> 4


## Dropping Redundant Columns

In [23]:
match_df = match_df[['team1' , 'team2','city','toss_decision','toss_winner','venue','winner']]
match_df

,team1,team2,city,toss_decision,toss_winner,venue,winner
0,10,3,Hyderabad,field,3,"Rajiv Gandhi International Stadium, Uppal",10
1,1,11,Pune,field,11,Maharashtra Cricket Association Stadium,11
2,8,2,Rajkot,field,2,Saurashtra Cricket Association Stadium,2
3,11,9,Indore,field,9,Holkar Cricket Stadium,9
4,3,7,Bangalore,bat,3,M Chinnaswamy Stadium,3
...,...,...,...,...,...,...,...
751,2,1,Mumbai,field,1,Wankhede Stadium,1
752,5,1,Chennai,bat,5,M. A. Chidambaram Stadium,1
753,10,15,Visakhapatnam,field,15,ACA-VDCA Stadium,15
754,15,5,Visakhapatnam,field,5,ACA-VDCA Stadium,5


In [24]:
from sklearn.preprocessing import LabelEncoder

ftr_list = ['city','toss_decision','venue']
encoder = LabelEncoder()

for f in ftr_list:
    match_df[f] = encoder.fit_transform(match_df[f])
    #print(encoder.classes_)

In [25]:
match_df

,team1,team2,city,toss_decision,toss_winner,venue,winner
0,10,3,15,1,3,28,10
1,1,11,27,1,11,21,11
2,8,2,29,1,2,31,2
3,11,9,16,1,9,13,9
4,3,7,2,0,3,17,3
...,...,...,...,...,...,...,...
751,2,1,24,1,1,40,1
752,5,1,8,0,5,18,1
753,10,15,32,1,15,0,15
754,15,5,32,1,5,0,5


# Machine Learning

In [53]:
from sklearn.model_selection import train_test_split

train_df,test_df = train_test_split(match_df,test_size = 0.2,random_state=5)

In [54]:
print(train_df.shape)
train_df

(604, 7)


,team1,team2,city,toss_decision,toss_winner,venue,winner
633,10,2,10,1,2,9,10
628,9,12,32,0,9,6,12
124,7,5,13,0,7,16,7
461,1,3,12,1,3,7,3
358,9,4,15,1,4,28,9
...,...,...,...,...,...,...,...
73,5,3,2,0,5,17,5
400,7,3,2,1,3,17,3
118,3,6,5,0,3,24,3
701,2,9,22,1,9,8,2


In [55]:
test_df.shape

(152, 7)

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [57]:
def print_model_score(model,train_data,test_data,predictor,target):
   
    #training the model
    model.fit(train_data[predictor],train_data[target])    
    
    #predicting on test_data
    prediction = model.predict(train_data[predictor])

    
    accuracy = accuracy_score(prediction,train_data[target])
    
    print('accuracy %s' % '{0:.2}'.format(accuracy))
    
    score = cross_val_score(model,train_data[predictor],train_data[target],scoring = "neg_mean_squared_error",cv=5)
    
    print("cross validation score : {}".format(np.sqrt(-score)))

In [58]:
target = ['winner']
predictor = ['team1','team2','venue','toss_winner','city','toss_decision']

lr = LogisticRegression()
print_model_score(lr,train_df,test_df,predictor,target)

accuracy 0.31
cross validation score : [3.6352271  3.55476633 3.54195632 3.67254725 3.73496988]


In [59]:
model = RandomForestClassifier(n_estimators=100)
print_model_score(model,train_df,test_df,predictor,target)

accuracy 0.89
cross validation score : [3.78011325 4.04008837 3.66127828 3.54428885 3.96127084]


In [72]:
pd.Series(index = predictor,data = model.feature_importances_)

team1            0.225878
team2            0.248008
venue            0.178716
toss_winner      0.164239
city             0.152345
toss_decision    0.030814
dtype: float64

In [92]:
team1 = 'Royal Challengers Bangalore'
team2 = 'Sunrisers Hyderabad'
toss_winner = 'Sunrisers Hyderabad'

#[team1, team2,city,toss winner,toss decision,venue]
inp = [team_encode_dict['team1'][team1],team_encode_dict['team2'][team2],'15',team_encode_dict['toss_winner'][toss_winner],'9','1']
inp = np.array(inp).reshape((1,-1))




In [93]:
output = model.predict(inp)

In [94]:
print(f"Winner : { list(team_encodings.keys())[list(team_encode_dict['team1'].values()).index(output)]}")

Winner : Royal Challengers Bangalore
